## Pancreatic Cancer Detection

### Dataset Creation

In [9]:
from dataset import Dataset
from pathlib import Path
import torch
from sklearn import preprocessing
import numpy as np
torch.manual_seed(94)
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
urinary_data = Dataset(filePath='../data/urinary_data.csv',
                       label_column='diagnosis',
                       separator=',', 
                       name='urinary_data')
display(urinary_data.dataframe.head())

,sample_id,patient_cohort,sample_origin,age,sex,diagnosis,stage,benign_sample_diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,S1,Cohort1,BPTB,33,F,1,NaN,NaN,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,S10,Cohort1,BPTB,81,F,1,NaN,NaN,NaN,0.97266,2.037585,94.46703,209.488250,228.407
2,S100,Cohort2,BPTB,51,M,1,NaN,NaN,7.0,0.78039,0.145589,102.36600,461.141000,NaN
3,S101,Cohort2,BPTB,61,M,1,NaN,NaN,8.0,0.70122,0.002805,60.57900,142.950000,NaN
4,S102,Cohort2,BPTB,62,M,1,NaN,NaN,9.0,0.21489,0.000860,65.54000,41.088000,NaN


### Dataset cleaning and preparation

In [11]:
# urinary_data.cleanDataframe()
# Encode labels
for column in urinary_data.dataframe.columns:
    urinary_data.encode_column(column)

display(urinary_data.dataframe.head())

# Dataset Cleaning
urinary_data.cleanDataframe()

display(urinary_data.dataframe.head())


,sample_id,patient_cohort,sample_origin,age,sex,diagnosis,stage,benign_sample_diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,0,0,0,6,0,0,8,52,78,158,181,337,391,247
1,1,0,0,54,0,0,8,52,266,93,265,402,242,151
2,2,1,0,24,1,0,8,52,62,75,108,412,347,298
3,3,1,0,34,1,0,8,52,67,68,47,351,209,298
4,4,1,0,35,1,0,8,52,70,16,14,360,117,298


Removed: 0 rows | 1 columns


,patient_cohort,sample_origin,age,sex,diagnosis,stage,benign_sample_diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,0,0,6,0,0,8,52,78,158,181,337,391,247
1,0,0,54,0,0,8,52,266,93,265,402,242,151
2,1,0,24,1,0,8,52,62,75,108,412,347,298
3,1,0,34,1,0,8,52,67,68,47,351,209,298
4,1,0,35,1,0,8,52,70,16,14,360,117,298


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Dataset split
X = urinary_data.dataframe.iloc[:, urinary_data.dataframe.columns != urinary_data.label_column].squeeze()
y = urinary_data.dataframe[urinary_data.label_column].values.reshape(-1, 1)
# Transform y to tensor of size equal to the number of classes
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
y = ohe.transform(y)


In [13]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Set sizes: Train: {}, Validation: {}, Test: {}".format(len(X_train), len(X_val), len(X_test)))

# Convert to tensors
X_train = torch.from_numpy(X_train.values).float().squeeze()
X_val = torch.from_numpy(X_val.values).float().squeeze()  
X_test = torch.from_numpy(X_test.values).float().squeeze()
y_train = torch.from_numpy(y_train).float().squeeze()
y_val = torch.from_numpy(y_val).float().squeeze()
y_test = torch.from_numpy(y_test).float().squeeze()


print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))


Set sizes: Train: 377, Validation: 95, Test: 118
X_train shape: torch.Size([377, 12])
y_train shape: torch.Size([377, 3])
X_val shape: torch.Size([95, 12])
y_val shape: torch.Size([95, 3])
X_test shape: torch.Size([118, 12])
y_test shape: torch.Size([118, 3])


###

In [14]:
# Define dictionary for label column
# 3 (pancreatic cancer), 2 (non-cancerous pancreas condition),  1 (healthy)
label_dict = {1: 'healthy', 2: 'non-cancerous pancreas condition', 3: 'pancreatic cancer'}
urinary_data.init_label_dictionary(label_column='diagnosis', label_dict=label_dict)

### Model Training

In [44]:
# Define loss function
# Binary classification => BCELoss
from models import train_binary_logits
from models import PCDModel_1
from models import MulticlassClassification
from models import accuracy_fn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt
# TRAINING
epochs = 1000000
model_0 = PCDModel_1(urinary_data.get_feature_count(), urinary_data.get_label_count())
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr=0.0001)

losses = []
for epoch in range(epochs):
    model_0.train()

    # 1. Forward pass
    y_logits = model_0(X_train).squeeze()
    y_pred = torch.sigmoid(y_logits)
    #print(y_pred.shape)
    # 2. Compute loss
    loss = loss_fn(y_pred, y_train)
    losses.append(loss)

    # 2.1 Compute accuracy
    acc = (torch.argmax(y_pred, 1) == torch.argmax(y_train, 1)).float().mean()


    # 3. Optimizer zero_grad
    optimizer.zero_grad()

    # 4. Backward pass
    loss.backward()

    # 5. Optimizer step
    optimizer.step()
    # 6. Test
    model_0.eval()
    with torch.inference_mode():
        # 6.1 Forward pass
        test_logits = model_0(X_test).squeeze() 
        test_pred = torch.sigmoid(test_logits)
        # 6.2 Compute loss
        test_loss = loss_fn(test_pred, y_test)
        # 6.3 Compute accuracy
        test_acc = (torch.argmax(test_pred, 1) == torch.argmax(y_test, 1)).float().mean()

    if epoch % 10 == 0:
        print("Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}".format(epoch, loss, acc, test_loss, test_acc))





Epoch: 0, Loss: 1.141438364982605, Accuracy: 0.42440319061279297, Test Loss: 1.1574848890304565, Test Accuracy: 0.41525423526763916
Epoch: 10, Loss: 1.0597195625305176, Accuracy: 0.32625994086265564, Test Loss: 1.0404177904129028, Test Accuracy: 0.43220338225364685
Epoch: 20, Loss: 1.0626325607299805, Accuracy: 0.3289124667644501, Test Loss: 1.0371259450912476, Test Accuracy: 0.3644067943096161
Epoch: 30, Loss: 1.060034990310669, Accuracy: 0.29708221554756165, Test Loss: 1.0351332426071167, Test Accuracy: 0.35593220591545105
Epoch: 40, Loss: 1.0583877563476562, Accuracy: 0.29708221554756165, Test Loss: 1.0351332426071167, Test Accuracy: 0.35593220591545105
Epoch: 50, Loss: 1.0583741664886475, Accuracy: 0.29708221554756165, Test Loss: 1.0351332426071167, Test Accuracy: 0.35593220591545105
Epoch: 60, Loss: 1.0583739280700684, Accuracy: 0.29708221554756165, Test Loss: 1.0351332426071167, Test Accuracy: 0.35593220591545105
Epoch: 70, Loss: 1.058372139930725, Accuracy: 0.29708221554756165, 

In [87]:
pred = model_0(X_test[10]).squeeze()
print(y_test[10])
print(pred)

tensor([0., 0., 1.])
tensor([-18.6728, -20.9609, -12.0012], grad_fn=<SqueezeBackward0>)
